In [1]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import requests
import pymongo
import pandas as pd
from flask import Flask,render_template,request,Response

In [2]:
app = Flask(__name__)


@app.route("/",methods=["GET"])
def home_page():
    return render_template("index.html")

@app.route("/scrap",methods=["POST","GET"])
def index():
    if (request.method=="POST"):
        SearchString = request.form["content"]
        try:
            url = "https://www.flipkart.com/search?q="+SearchString
            client = urlopen(url)
            page = client.read()
            client.close()
            page_html = bs(page,"html.parser")
            bigbox = page_html.findAll("div",{"class":"_2pi5LC col-12-12"})
            del bigbox[:2]
            big = bigbox[0]
            product_link = "https://www.flipkart.com"+big.div.div.div.a["href"]
            client2 = urlopen(product_link)
            page2 = client2.read()
            client2.close()
            page2_html = bs(page2,"html.parser")
            commentboxes = page2_html.findAll("div",{"class":"_16PBlm"})
            del commentboxes[-1]
            review = []
            for box in commentboxes:
                try:
                    rating = box.div.div.div.div.text
                except:
                    print("NO RATING")
                try:
                    rew = box.div.div.findAll("div",{"class":""})[0].text
                except:
                    print("NO REVIEW")
                try:
                    name = box.div.findAll("p",{"class":"_2sc7ZR _2V5EHH"})[0].text
                except:
                    print("NO NAME")
                
                dictionary = {"NAME":name,"RATING":rating,"REVIEW":rew}
                
                review.append(dictionary)
            return render_template("results.html", reviews=review)
                
                
                     
        except Exception as e:
            print(e)
    else:
        return render_template("index.html")
            


if (__name__) == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2020-11-19 12:47:37,852] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\saran\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\saran\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\saran\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\saran\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\saran\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\saran\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-2-0dca3